<style>
    body, * {
        direction: rtl !important;
        text-align: right !important;
    }
</style>
# 📦 PySpark - קריאה וכתיבה של קובץ Parquet
[PySpark Read and Write Parquet File](https://sparkbyexamples.com/pyspark/pyspark-read-and-write-parquet-file/)
## מבוא

Pyspark SQL מספק שיטות לקריאה של Parquet file ל-DataFrame וכתיבה ל-Parquet file, `parquet()` מה-DataFrameReader ו-DataFrameWriter משמשים לקריאה מקובץ Parquet ל-DataFrame וכתיבה ל-Parquet file בהתאמה.

Parquet files שומרים את הסכימה יחד עם הנתונים ומכאן שהוא משמש לעתים קרובות לעיבוד structured files.

במאמר זה, אסביר כיצד לכתוב parquet file מ-DataFrame ב-PySpark, כיצד לקרוא, וגם אסביר כיצד לעשות partitions על parquet files כדי לשפר את הביצועים.

---

## 📋 מה זה Parquet File?

Apache Parquet file הוא פורמט אחסון columnar זמין לכל פרויקט ב-Hadoop ecosystem, ללא קשר לבחירת framework עיבוד הנתונים, מודל הנתונים או שפת התכנות.

### 🎯 יתרונות:

בזמן שאילתת אחסון columnar, הוא מדלג על נתונים שאינם רלוונטיים מאוד במהירות, מה שהופך את ביצוע ה-aggregation queries מהיר יותר. כתוצאה מכך, aggregation queries צורכות פחות זמן בהשוואה ל-row-oriented databases.

הוא מסוגל לתמוך בcompression options ו-encoding schemes מתקדמים.

---

## 🔧 דוגמה ל-Apache Parquet Pyspark

מכיוון שאין לנו את קובץ ה-parquet, בואו ניצור DataFrame מרשימת נתונים באמצעות `spark.createDataFrame()` method.

In [ ]:
# יצירת נתונים לדוגמה
# ייבואים
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("parquetFile").getOrCreate()

data = [("James", "", "Smith", "36636", "M", 3000),
        ("Michael", "Rose", "", "40288", "M", 4000),
        ("Robert", "", "Williams", "42114", "M", 4000),
        ("Maria", "Anne", "Jones", "39192", "F", 4000),
        ("Jen", "Mary", "Brown", "", "F", -1)
        ]

columns = ["firstname", "middlename", "lastname", "dob", "gender", "salary"]
df = spark.createDataFrame(data=data, schema=columns)
df.show()
df.printSchema()

זה יוצר DataFrame עם עמודות `firstname`, `middlename`, `lastname`, `dob`, `gender`, `salary` ובאופן דיפולטי כל סוגי הנתונים של העמודות מטופלים כ-String.

---

## 💾 Pyspark Write DataFrame לקובץ Parquet

עכשיו בואו נכתוב את ה-DataFrame שיצרנו ל-parquet file באמצעות קריאה ל-`parquet()` מה-DataFrameWriter class.

In [ ]:
# כתיבת DataFrame לפורמט parquet באמצעות write.parquet()
df.write.parquet("tmp/02_parquet/people.parquet")


זה יוצר parquet file ב-parquet format. כאשר אתה מריץ את התוכנית הזו מ-Spyder IDE, הוא יוצר `people.parquet` file ו-`spark-warehouse` directory תחת התיקייה הנוכחית.

---

## 📖 Pyspark Read Parquet file ל-DataFrame

Pyspark מספק שיטת `parquet()` ב-DataFrameReader class כדי לקרוא את קובץ ה-parquet ל-dataframe. בדוגמה למטה זה קורא את קובץ ה-parquet שיצרנו בסעיף הקודם.

In [ ]:
# קריאת parquet file באמצעות read.parquet()
parDF = spark.read.parquet("tmp/02_parquet/people.parquet")

---

## 📊 סוגי Saving Modes של Parquet File

ב-PySpark, לאחר כתיבת ה-DataFrame ל-Parquet file, אנחנו יכולים לציין את mode של השמירה באמצעות שיטת `.mode()` מה-pyspark.sql.DataFrameWriter.mode module.

### תחביר
```
DataFrameWriter.mode(saveMode: Optional[str])
```

### אפשרויות:

**append**: mode זה מוסיף את הנתונים ל-DataFrame מה-file הקיים, אם קבצי היעד לא קיימים, הוא יוצר parquet file חדש.

דוגמה:

In [ ]:
# df.write.mode("append").parquet("/path/to/parquet/file")

**overwrite**: mode זה דורס את קובץ ה-Parquet היעד עם הנתונים מה-DataFrame. אם הקובץ לא קיים, הוא יוצר Parquet file חדש.

דוגמה:

In [ ]:
# df.write.mode("overwrite").parquet("/path/to/parquet/file")

**ignore**: אם קובץ Parquet היעד כבר קיים, mode זה לא עושה כלום ואינו כותב את ה-DataFrame לקובץ. אם הקובץ לא קיים, הוא יוצר Parquet file חדש.

דוגמה:

In [ ]:
# df.write.mode("ignore").parquet("/path/to/parquet/file")

**error או errorifexists**: זה זורק error אם קובץ Parquet היעד כבר קיים. זו ההתנהגות הדיפולטית אם לא צוין saving mode.

דוגמה:

In [ ]:
# df.write.mode("error").parquet("/path/to/parquet/file")

---

## 🔍 ביצוע SQL queries על Parquet file

Pyspark Sql מספק ליצור temporary view על parquet files כדי לבצע sql queries.

קריאה של parquet files מאפשרת ליצור views/tables זמניות באמצעות `createOrReplaceTempView()`.

In [ ]:
# שימוש ב-spark sql
parDF.createOrReplaceTempView("ParquetTable")
parkSQL = spark.sql("select * from ParquetTable where salary >= 4000 ")
parkSQL.show(truncate=False)

---

## 📁 יצירת Parquet partition file

כשאנחנו מבצעים query מסוים על parquet table, אנחנו סורקים את כל השורות וכל הpartitions בטבלה, זה נתפס בעייתי כאשר יש לנו partitions רבות ב-Hive. זה לא ברור מאוד בדוגמה הנוכחית כי יש לנו רק שתי partitions "gender" followed by "salary".

השתמש ב-`partitionBy()` של pyspark.sql.DataFrameWriter כדי ליצור parquet file בדרך מחולקת.

In [ ]:
df.write.partitionBy("gender", "salary") \
    .mode("overwrite") \
    .parquet("tmp/02_parquet/people2.parquet")

כשאתה בודק את people2.parquet file, אתה רואה partitions "gender" followed by "salary" בתוכה.

### תוכן התיקייה:

```
tmp > 02_parquet > people2.parquet
  gender=F
    salary=-1
    salary=4000
  gender=M
    salary=3000
    salary=4000
```

---

## 🎯 קריאה מ-Partitioned Parquet file

הדוגמה למטה מסבירה קריאת partitioned parquet file ל-DataFrame עם gender=M.

In [ ]:
parDF2 = spark.read.parquet("tmp/02_parquet/people2.parquet/gender=M")
parDF2.show(truncate=False)

---

## 🏷️ יצירת table על Partitioned Parquet file

כאן, אני יוצר temporary view על partitioned parquet file ומבצע query שמבצע פעולות מהירות יותר מהטבלה ללא partitioning, ומכאן משפר את הביצועים.

In [ ]:
# יצירת temporary view על Parquet File מחולק
spark.sql("CREATE TEMPORARY VIEW PERSON USING parquet OPTIONS (path \"tmp/02_parquet/people2.parquet\")")
spark.sql("SELECT * FROM PERSON").show()

df.write.partitionBy("gender", "salary") \
    .mode("overwrite") \
    .parquet("tmp/02_parquet/people2.parquet")

spark.sql("CREATE TEMPORARY VIEW PERSON2 USING parquet OPTIONS (path \"tmp/02_parquet/people2.parquet\")")
spark.sql("SELECT * FROM PERSON2 ").show()

---

## 📝 דוגמה מלאה של PySpark read וכתיבת Parquet file

In [ ]:
# ייבואים
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("parquetFile").getOrCreate()

data = [("James", "", "Smith", "36636", "M", 3000),
        ("Michael", "Rose", "", "40288", "M", 4000),
        ("Robert", "", "Williams", "42114", "M", 4000),
        ("Maria", "Anne", "Jones", "39192", "F", 4000),
        ("Jen", "Mary", "Brown", "", "F", -1)
        ]

columns = ["firstname", "middlename", "lastname", "dob", "gender", "salary"]
df = spark.createDataFrame(data=data, schema=columns)
df.show()
df.printSchema()

df.write.mode("overwrite").parquet("tmp/02_parquet/people.parquet")

parDF1 = spark.read.parquet("tmp/02_parquet/people.parquet")
parDF1.createOrReplaceTempView("ParquetTable")
parDF1.printSchema()
parDF1.show(truncate=False)

parkSQL = spark.sql("select * from ParquetTable where salary >= 4000 ")
parkSQL.show(truncate=False)

spark.sql("CREATE TEMPORARY VIEW PERSON3 USING parquet OPTIONS (path \"tmp/02_parquet/people.parquet\")")
spark.sql("SELECT * FROM PERSON3").show()

df.write.partitionBy("gender", "salary") \
    .mode("overwrite") \
    .parquet("tmp/02_parquet/people2.parquet")

parDF2 = spark.read.parquet("tmp/02_parquet/people2.parquet/gender=M")
parDF2.show(truncate=False)

spark.sql("CREATE TEMPORARY VIEW PERSON4 USING parquet OPTIONS (path \"tmp/02_parquet/people2.parquet\")")
spark.sql("SELECT * FROM PERSON4").show()

---

## 🎓 סיכום:

למדנו כיצד לכתוב parquet file מ-PySpark DataFrame וקריאה של parquet files, ליצור views/tables ל-parquet files ועוד נושאים קשורים.

תקווה שאהבת את זה, השאירו תגובה בקטע התגובות.

### 💡 למידה מהנה!!

---

## 📚 מאמרים קשורים

- PySpark Shell Command Usage with Examples
- PySpark Retrieve DataType & Column Names of DataFrame
- PySpark Parse JSON from String Column | TEXT File
- PySpark SQL Types (DataType) with Examples
- PySpark SparkContext Explained
- Explain PySpark inferSchema
- PySpark Schema Strategies: When to Use inferSchema, mergeSchema, and overwriteSchema